# Siddon's Method

> Mapping from ray to pixel intensity

In [ ]:
#| default_exp siddon

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch

## DRR Generation

The process of generating a DRR models the geometry of an idealized projectional radiography system.
Let $\mathbf s \in \mathbb R^3$ be the X-ray source, and $\mathbf p \in \mathbb R^3$ be a target pixel on the detector plane.
Then $R(\alpha) = \mathbf s + \alpha (\mathbf p - \mathbf s)$ is a ray that originates from $\mathbf s$ ($\alpha=0$), passes through the imaged volume, and hits the detector plane at $\mathbf p$ ($\alpha=1$).
The total energy attenuation experienced by the X-ray by the time it reaches pixel $\mathbf p$ is given by the following line integral:

\begin{equation}
    E(R) = \|\mathbf p - \mathbf s\|_2 \int_0^1 \mathbf V \left( \mathbf s + \alpha (\mathbf p - \mathbf s) \right) \mathrm d\alpha \,,
\end{equation}

where $\mathbf V : \mathbb R^3 \mapsto \mathbb R$ is the imaged volume.
The term $\|\mathbf p - \mathbf s\|_2$ endows the unit-free $\mathrm d \alpha$ with the physical unit of length.
For DRR synthesis, $\mathbf V$ is approximated by a discrete 3D CT volume, and Eq. (1) becomes

\begin{equation}
    E(R) = \|\mathbf p - \mathbf s\|_2 \sum_{m=1}^{M-1} (\alpha_{m+1} - \alpha_m) \mathbf V \left[ \mathbf s + \frac{\alpha_{m+1} + \alpha_m}{2} (\mathbf p - \mathbf s) \right] \,,
\end{equation}

where $\alpha_m$ parameterizes the locations where ray $R$ intersects one of the orthogonal planes comprising the CT volume, and $M$ is the number of such intersections.
Note that this model does not account for patterns of reflection and scattering that are present in real X-ray systems.
While these simplifications preclude synthesis of realistic X-rays, the model in Eq. (2) has been widely and successfully used in slice-to-volume registration.
Additionally, our approach of vectorizing DRR generation might also be interoperable with more sophisticated image synthesis models, an extension we examine further in the Discussion.

In [ ]:
#| export
def siddon_raycast(
    source: torch.Tensor,
    target: torch.Tensor,
    volume: torch.Tensor,
    spacing: torch.Tensor,
    eps: float=1e-8,
):
    """Compute Siddon's method."""
    dims = torch.tensor(volume.shape) + 1
    alphas, maxidx = _get_alphas(source, target, spacing, dims, eps)
    alphamid = (alphas[..., 0:-1] + alphas[..., 1:]) / 2
    voxels = _get_voxel(alphamid, source, target, volume, spacing, dims, maxidx, eps)

    # Step length for alphas out of range will be nan
    # These nans cancel out voxels convereted to 0 index
    step_length = torch.diff(alphas, dim=-1)
    weighted_voxels = voxels * step_length

    drr = torch.nansum(weighted_voxels, dim=-1)
    raylength = (target - source + eps).norm(dim=-1)
    drr *= raylength
    return drr

Siddon's method provides a parametric method to identify the plane intersections $\{\alpha_m\}_{m=1}^M$.
Let $\Delta X$ be the CT voxel size in the $x$-direction and $b_x$ be the location of the $0$-th plane in this direction.
Then the intersection of ray $R$ with the $i$-th plane in the $x$-direction is given by
\begin{equation}
    \label{eqn:x-intersect}
    \alpha_x(i) = \frac{b_x + i \Delta X - \mathbf s_x}{\mathbf p_x - \mathbf s_x} ,
\end{equation}
with analogous expressions for $\alpha_y(\cdot)$ and $\alpha_z(\cdot)$.

We can use Eq. (3) to compute the values $\mathbf \alpha_x$ for all the intersections between $R$ and the planes in the $x$-direction:
\begin{equation*}
    \mathbf\alpha_x = \{ \alpha_x(i_{\min}), \dots, \alpha_x(i_{\max}) \} ,
\end{equation*}
where $i_{\min}$ and $i_{\max}$ denote the first and last intersections of $R$ with the $x$-direction planes.

Defining $\mathbf\alpha_y$ and $\mathbf\alpha_z$ analogously, we construct the array
\begin{equation}
    \label{eqn:alphas}
    \mathbf\alpha = \mathrm{sort}(\mathbf\alpha_x, \mathbf\alpha_y, \mathbf\alpha_z) ,
\end{equation}
which contains $M$ values of $\alpha$ parameterizing the intersections between $R$ and the orthogonal $x$-, $y$-, and $z$-directional planes. 
We substitute values in the sorted set $\mathbf\alpha$ into Eq. (2) to evaluate $E(R)$, which corresponds to the intensity of pixel $\mathbf p$ in the synthesized DRR.

In [ ]:
#| export
def _get_alphas(source, target, spacing, dims, eps):
    # Get the CT sizing and spacing parameters
    dx, dy, dz = spacing
    nx, ny, nz = dims
    maxidx = ((nx - 1) * (ny - 1) * (nz - 1)).int().item() - 1

    # Get the alpha at each plane intersection
    sx, sy, sz = source[..., 0], source[..., 1], source[..., 2]
    alphax = torch.arange(nx).to(source) * dx
    alphay = torch.arange(ny).to(source) * dy
    alphaz = torch.arange(nz).to(source) * dz
    alphax = alphax.expand(len(source), 1, -1) - sx.unsqueeze(-1)
    alphay = alphay.expand(len(source), 1, -1) - sy.unsqueeze(-1)
    alphaz = alphaz.expand(len(source), 1, -1) - sz.unsqueeze(-1)

    sdd = target - source + eps
    alphax = alphax / sdd[..., 0].unsqueeze(-1)
    alphay = alphay / sdd[..., 1].unsqueeze(-1)
    alphaz = alphaz / sdd[..., 2].unsqueeze(-1)
    alphas = torch.cat([alphax, alphay, alphaz], dim=-1)

    # # Get the alphas within the range [alphamin, alphamax]
    alphamin, alphamax = _get_alpha_minmax(source, target, spacing, dims, eps)
    good_idxs = torch.logical_and(alphas >= alphamin, alphas <= alphamax)
    alphas[~good_idxs] = torch.nan

    # # Sort the alphas by ray, putting nans at the end of the list
    # # Drop indices where alphas for all rays are nan
    alphas = torch.sort(alphas, dim=-1).values
    alphas = alphas[..., ~alphas.isnan().all(dim=0).all(dim=0)]
    return alphas, maxidx


def _get_alpha_minmax(source, target, spacing, dims, eps):
    sdd = target - source + eps
    planes = torch.zeros(3).to(source)
    alpha0 = (planes * spacing - source) / sdd
    planes = (dims - 1).to(source)
    alpha1 = (planes * spacing - source) / sdd
    alphas = torch.stack([alpha0, alpha1]).to(source)

    alphamin = alphas.min(dim=0).values.max(dim=-1).values.unsqueeze(-1)
    alphamax = alphas.max(dim=0).values.min(dim=-1).values.unsqueeze(-1)
    return alphamin, alphamax


def _get_voxel(alpha, source, target, volume, spacing, dims, maxidx, eps):
    idxs = _get_index(alpha, source, target, spacing, dims, maxidx, eps)
    return torch.take(volume, idxs)


def _get_index(alpha, source, target, spacing, dims, maxidx, eps):
    sdd = target - source + eps
    idxs = source.unsqueeze(1) + alpha.unsqueeze(-1) * sdd.unsqueeze(2)
    idxs = idxs / spacing
    idxs = idxs.trunc()
    # Conversion to long makes nan->-inf, so temporarily replace them with 0
    # This is cancelled out later by multiplication by nan step_length
    idxs = (
        idxs[..., 0] * (dims[1] - 1) * (dims[2] - 1)
        + idxs[..., 1] * (dims[2] - 1)
        + idxs[..., 2]
    ).long() + 1
    idxs[idxs < 0] = 0
    idxs[idxs > maxidx] = maxidx
    return idxs

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()